In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# 라이브러리 선언
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [3]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [4]:
# 원하는 데이터 구조를 정의합니다.
class Filter(BaseModel):
    NATURAL_LANGUAGE: str = Field(description="사용자 검색어를 자연스러운 문장으로 변환.")
    SALE_PRC_GTE: int = Field(description="가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000")
    SALE_PRC_LTE: int = Field(description="가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000")
    BRND_NM: str = Field(description="브랜드명")
    ARTC_NM: str = Field(description="품목")
    CATEGORY_NM: str = Field(description="""카테고리명. 다음 카테고리 예시 중 반드시 1개. 
안심케어
전문가 화상상담
1인 가구를 위한 나노스퀘어 
TV·영상가전
가구·인테리어
태블릿·이북리더기
건강가전
게임기·타이틀
냉장고·주방가전
레저·여행·헬스케어
렌탈·구독
휴대폰·스마트워치
생활·주방용품
세탁기·건조기·의류관리기
에어컨·계절가전
청소기·생활가전
카메라
컴퓨터·노트북
스마트홈
음향가전
뷰티·이미용가전
문구·악기·공구
가전 수리비 보장 가전보험
방문컨설팅""")
    FEATURES: str = Field(description="주요기능")

# {'SALE_PRC': {'$gte': 1000000, '$lte': 1200000}}

In [5]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Filter)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"NATURAL_LANGUAGE": {"description": "사용자 검색어를 자연스러운 문장으로 변환.", "title": "Natural Language", "type": "string"}, "SALE_PRC_GTE": {"description": "가격범위 최소값. 예를 들어 100만원 이상 120만원 이하이면, 1000000", "title": "Sale Prc Gte", "type": "integer"}, "SALE_PRC_LTE": {"description": "가격범위 최대값. 예를 들어 100만원 이상 120만원 이하이면, 1200000", "title": "Sale Prc Lte", "type": "integer"}, "BRND_NM": {"description": "브랜드명", "title": "Brnd Nm", "type": "string"}, "ARTC_NM": {"description": "품목", "title": "Artc Nm", "type": "string"}, "CATEGORY_NM": {"descripti

In [6]:

# 프롬프트를 생성합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 가전전문 e커머스 검색 AI 어시스턴트 입니다. 사용자의 검색어를 분석하여, 그 의도에 맞는 최적의 검색 필터 조건을 완성해야 합니다."),
        ("user", "#Format: {format_instructions}\n\n#Question: {query}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성합니다.
chain = prompt | model | parser

In [7]:
# 체인을 호출하여 쿼리 실행
query = "아기용 가습기"
answer = chain.invoke({"query": query})
print(f"{query}: {answer}")

# query = "5만원 이상 블루투스 마우스"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "4인 가족 냉장고 추천"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "원룸용 세탁기"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "게이밍용 고성능 모니터"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "신혼부부용 가전세트"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "국산 로봇청소기 추천"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "4K HDR 지원 55인치 TV"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "20만원대 스팀 다리미"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "인덕션 3구 전기레인지"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "조용하고 전기요금 적게 나오는 에어컨"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")

# query = "자동급수 가능한 로봇청소기"
# answer = chain.invoke({"query": query})
# print(f"{query}: {answer}")




아기용 가습기: {'NATURAL_LANGUAGE': '아기용 가습기를 찾습니다', 'SALE_PRC_GTE': 0, 'SALE_PRC_LTE': 1000000, 'BRND_NM': '', 'ARTC_NM': '가습기', 'CATEGORY_NM': '생활·주방용품', 'FEATURES': '아기용'}
